<a href="https://colab.research.google.com/github/tanoManzo/mimic_attitude/blob/dev/full_mimic_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Testing
Testing models after training and tuning 

necessary libraries

In [ ]:
!pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 25.2 MB/s eta 0:00:00


In [ ]:
! pip install datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 20.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from transformers import pipeline # E2E automized 
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch 
import torch.nn.functional as F # toolbox: softmax, sigmoid, ReLu


load the models from Hugging Face hub

In [ ]:
# model name from huggingface.co/model name_id:model_name
models_name = {
  'roberta':'tanoManzo/roberta-attitude',
  'distilbert':'tanoManzo/distilbert-attitude',
  'minilm':'tanoManzo/minilm-attitude',
  'bloom':'tanoManzo/bloom-attitude'
}

# load tokenizer and model 
tokenizers = [AutoTokenizer.from_pretrained(model_name) for model_name in models_name.values()]
models= [AutoModelForSequenceClassification.from_pretrained(model_name) for model_name in models_name.values()]

load test set from Hugging Face (token required, email: gaetano.manzo@nih.gov)

In [ ]:
from huggingface_hub import notebook_login
from datasets import load_dataset
notebook_login()
data = load_dataset('tanoManzo/mimic_attitude_train_val_test', use_auth_token=True)

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label'],
        num_rows: 133
    })
    test: Dataset({
        features: ['id', 'text', 'label'],
        num_rows: 45
    })
    validation: Dataset({
        features: ['id', 'text', 'label'],
        num_rows: 45
    })
})

get ready for testing

In [ ]:
# select dataframe split
df_to_infer = data['test'].to_pandas()

# parameters to set 
total_notes = len(df_to_infer)
batch_size = 5
iterations = int(len(df_to_infer)/batch_size)

testing 

In [ ]:
l_scores = {k:[] for k in models_name.keys()} # dict Model: labels 
l_labels = {k:[] for k in models_name.keys()} # dict Model: scoress
for iter in range(iterations):
  

  idx_start = batch_size*iter     # first window sentence idx in the dataframe
  idx_end = batch_size*(iter+1)   # last window sentence idx in the dataframe
  
  # get row sentences for batch
  sentences = list(df_to_infer['text'].iloc[idx_start:idx_end].values)
 
  for (index, (key, value)) in enumerate(models_name.items()):
    
    # tokenized input for the model 
    tokenizer = tokenizers[index]
    batch = tokenizer(sentences, # list of sentence 
                    padding=True, # add if short 
                    truncation=True, # remove if long 
                    max_length=512, # sent length
                    return_tensors="pt" # to return pytorch tensor (NO for TF)
                    )
  
  
    
    model = models[index]
    with torch.no_grad():
      outputs = model(**batch)
      predictions = F.softmax(outputs.logits, dim=1)
      label_ids = torch.argmax(predictions, dim=1)

      labels = [model.config.id2label[label_id] for label_id in label_ids.tolist()]
      scores = [round(float(predictions[idx][label_id].item()),4) for idx, label_id in enumerate(label_ids.tolist())]
      l_labels[key].extend(labels)
      l_scores[key].extend(scores)

results

In [ ]:
l_labels, l_scores

({'roberta': ['Overall Negative Note',
   'Overall Neutral Note',
   'Overall Neutral Note',
   'Overall Neutral Note',
   'Overall Neutral Note',
   'Overall Neutral Note',
   'Overall Neutral Note',
   'Overall Neutral Note',
   'Overall Neutral Note',
   'Overall Neutral Note',
   'Overall Neutral Note',
   'Overall Neutral Note',
   'Overall Neutral Note',
   'Overall Negative Note',
   'Overall Neutral Note',
   'Overall Neutral Note',
   'Overall Neutral Note',
   'Overall Negative Note',
   'Overall Neutral Note',
   'Overall Neutral Note',
   'Overall Neutral Note',
   'Overall Neutral Note',
   'Overall Neutral Note',
   'Overall Neutral Note',
   'Overall Neutral Note',
   'Overall Neutral Note',
   'Overall Neutral Note',
   'Overall Neutral Note',
   'Overall Negative Note',
   'Overall Neutral Note',
   'Overall Neutral Note',
   'Overall Neutral Note',
   'Overall Neutral Note',
   'Overall Neutral Note',
   'Overall Neutral Note',
   'Overall Neutral Note',
   'Overall N

Save results for evaluation

In [ ]:
import json


json1 = json.dumps(l_labels)
# open file for writing, "w" 
f = open("/content/drive/MyDrive/NIH/Data/results_burnout/label_full.json","w")
# write json object to file
f.write(json1)


json2 = json.dumps(l_scores)
# open file for writing, "w" 
f = open("/content/drive/MyDrive/NIH/Data/results_burnout/score_full.json","w")
# write json object to file
f.write(json2)

1384